In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
CFG = {
    "csv_path":      "mumbai_weekly_tariffs.csv",
    "battery_kwh":   13.5,
    "battery_kw":    5.0,
    "eta_round":     0.95,
    "soc_initial":   0.3,
    "house_load_kw": 0.8,
    "plot_hours":    168,
    "PLOTTING":      False,
}

In [4]:
def read_mumbai_csv(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df["date"] = pd.to_datetime(df["Date"])
    df["start_hour"] = df["Time_Slot"].str.split("-").str[0].str[:2].astype(int)
    df["timestamp"] = df["date"] + pd.to_timedelta(df["start_hour"], unit="h")
    hourly = []
    for _, r in df.iterrows():
        idx = pd.date_range(r["timestamp"], freq="H", periods=6, tz="Asia/Kolkata")
        hourly.append(pd.DataFrame({"timestamp": idx, "tariff": r["Tariff_(₹/kWh)"]}))
    dfh = pd.concat(hourly).drop_duplicates("timestamp").sort_values("timestamp")
    return dfh.set_index("timestamp")

In [5]:
def plan_hour(tariff: float, load_kw: float,
              soc_prev: float, bat_kwh: float, bat_kw: float, eta: float,
              threshold: float) -> dict:
    if tariff >= threshold:          # EXPENSIVE → discharge
        max_discharge = min(bat_kw, soc_prev * bat_kwh)
        discharge_kw  = min(max_discharge, load_kw / eta)
        grid_kw   = max(0.0, load_kw - discharge_kw * eta)
        battery_kw = discharge_kw
        soc_new    = soc_prev - discharge_kw / bat_kwh
        mode       = "discharge"
    else:                            # CHEAP → charge
        max_charge = min(bat_kw, (1 - soc_prev) * bat_kwh)
        battery_kw = -max_charge
        grid_kw    = load_kw + max_charge
        soc_new    = soc_prev + max_charge * eta / bat_kwh
        mode       = "charge"
    return {"grid_kw": grid_kw,
            "battery_kw": battery_kw,
            "soc_new": np.clip(soc_new, 0, 1),
            "mode": mode}

In [6]:
def simulate(df_future: pd.DataFrame, cfg: dict) -> pd.DataFrame:
    records, soc = [], cfg["soc_initial"]
    threshold = df_future["tariff"].mean()
    print(f"Dynamic threshold (future mean): {threshold:.2f} ₹/kWh")
    for ts, row in df_future.iterrows():
        res = plan_hour(row["tariff"], cfg["house_load_kw"],
                        soc, cfg["battery_kw"], cfg["battery_kw"], cfg["eta_round"],
                        threshold)
        records.append({"timestamp": ts, "tariff": row["tariff"], **res})
        soc = res["soc_new"]
    return pd.DataFrame(records).set_index("timestamp")

In [ ]:
df_all    = read_mumbai_csv(CFG["csv_path"])

test_day = df_all.index.normalize().unique()[0]
df_future = df_all[df_all.index.normalize() == test_day].copy()
print(f"Running simulation on test day: {test_day.date()}")

schedule = simulate(df_future, CFG)
print("Simulation finished – first 5 rows:")
display(schedule.head())

Running simulation on test day: 2025-10-26
Dynamic threshold (future mean): 6.09 ₹/kWh
Simulation finished – first 5 rows:


C:\Users\HP\AppData\Local\Temp\ipykernel_50480\1785266983.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  idx = pd.date_range(r["timestamp"], freq="H", periods=6, tz="Asia/Kolkata")


,tariff,grid_kw,battery_kw,soc_new,mode
timestamp,,,,,
2025-10-26 00:00:00+05:30,4.5,4.300000,-3.500000,0.965000,charge
2025-10-26 01:00:00+05:30,4.5,0.975000,-0.175000,0.998250,charge
2025-10-26 02:00:00+05:30,4.5,0.808750,-0.008750,0.999912,charge
2025-10-26 03:00:00+05:30,4.5,0.800438,-0.000438,0.999996,charge
2025-10-26 04:00:00+05:30,4.5,0.800022,-0.000022,1.000000,charge


In [9]:
if CFG["PLOTTING"]:
    import matplotlib.pyplot as plt
    sub = schedule.head(CFG["plot_hours"])
    sub["tariff"].plot(figsize=(14,3), title="Hourly tariff")
    plt.ylabel("₹/kWh")
    plt.tight_layout()
    plt.show()

In [10]:
def build_deployment_schedule(df_future: pd.DataFrame, cfg: dict) -> pd.DataFrame:
    records, soc = [], cfg["soc_initial"]
    threshold = df_future["tariff"].mean()
    for ts, row in df_future.iterrows():
        if row["tariff"] >= threshold and soc > 0:   # expensive & energy left
            max_discharge = min(cfg["battery_kw"], soc * cfg["battery_kwh"])
            discharge_kw  = min(max_discharge, cfg["house_load_kw"] / cfg["eta_round"])
            grid_kw       = max(0.0, cfg["house_load_kw"] - discharge_kw * cfg["eta_round"])
            battery_kw    = discharge_kw
            soc_new       = soc - discharge_kw / cfg["battery_kwh"]
            mode          = "discharge"
        else:                                        # cheap or empty → no discharge
            grid_kw    = cfg["house_load_kw"]
            battery_kw = 0.0
            soc_new    = soc
            mode       = "idle"
        records.append({"timestamp": ts, "tariff": row["tariff"],
                        "grid_kw": grid_kw, "battery_kw": battery_kw,
                        "soc_new": np.clip(soc_new, 0, 1), "mode": mode})
        soc = soc_new
    return pd.DataFrame(records).set_index("timestamp")

deployment_schedule = build_deployment_schedule(df_future, CFG)
print("Deployment schedule (first 5 rows):")
display(deployment_schedule.head())

Deployment schedule (first 5 rows):


,tariff,grid_kw,battery_kw,soc_new,mode
timestamp,,,,,
2025-10-26 00:00:00+05:30,4.5,0.8,0.0,0.3,idle
2025-10-26 01:00:00+05:30,4.5,0.8,0.0,0.3,idle
2025-10-26 02:00:00+05:30,4.5,0.8,0.0,0.3,idle
2025-10-26 03:00:00+05:30,4.5,0.8,0.0,0.3,idle
2025-10-26 04:00:00+05:30,4.5,0.8,0.0,0.3,idle


In [ ]:
def build_charging_schedule(deploy_df: pd.DataFrame, cfg: dict) -> pd.DataFrame:
    daily_deficit = (deploy_df["battery_kw"] / cfg["eta_round"]).resample("D").sum()
    charge_df = deploy_df.copy()
    charge_df["battery_kw"] = 0.0
    charge_df["grid_kw"]    = cfg["house_load_kw"]
    charge_df["mode"]       = "idle"

    for day, deficit in daily_deficit.items():
        if deficit <= 0:
            continue
        day_mask   = charge_df.index.date == day
        day_hours  = charge_df.loc[day_mask].copy()
        cheapest = day_hours.sort_values("tariff").index
        energy_needed = deficit
        for ts in cheapest:
            if energy_needed <= 0:
                break
            max_charge_energy = min(cfg["battery_kw"], energy_needed)
            charge_df.loc[ts, "battery_kw"] = -max_charge_energy   
            charge_df.loc[ts, "grid_kw"]   += max_charge_energy    
            charge_df.loc[ts, "mode"]       = "charge"
            energy_needed -= max_charge_energy
    return charge_df

charging_schedule = build_charging_schedule(deployment_schedule, CFG)
print("Charging schedule (first 5 rows):")
display(charging_schedule.head())

Charging schedule (first 5 rows):


,tariff,grid_kw,battery_kw,soc_new,mode
timestamp,,,,,
2025-10-26 00:00:00+05:30,4.5,0.8,0.0,0.3,idle
2025-10-26 01:00:00+05:30,4.5,0.8,0.0,0.3,idle
2025-10-26 02:00:00+05:30,4.5,0.8,0.0,0.3,idle
2025-10-26 03:00:00+05:30,4.5,0.8,0.0,0.3,idle
2025-10-26 04:00:00+05:30,4.5,0.8,0.0,0.3,idle


In [12]:
if CFG["PLOTTING"]:
    sub_d = deployment_schedule.head(CFG["plot_hours"])
    sub_c = charging_schedule.head(CFG["plot_hours"])
    ax = sub_d["battery_kw"].plot(kind="step", figsize=(14,3), label="Deployment (discharge)")
    sub_c["battery_kw"].plot(kind="step", ax=ax, label="Charging (max rate)", color="C2")
    ax.axhline(0, color="grey", lw=0.8)
    ax.set_ylabel("kW")
    ax.set_title("Separate deployment & charging schedules")
    plt.tight_layout()
    plt.show()

In [13]:
deployment_schedule.to_csv("deployment_schedule.csv")
charging_schedule.to_csv("charging_schedule.csv")
print("Saved →", Path("deployment_schedule.csv").resolve())
print("Saved →", Path("charging_schedule.csv").resolve())

Saved → C:\Users\HP\Documents\Codes\Enervia\deployment_schedule.csv
Saved → C:\Users\HP\Documents\Codes\Enervia\charging_schedule.csv


In [ ]:
deployment_schedule["daily_grid_only"] = (deployment_schedule["tariff"] *
                                          CFG["house_load_kw"])
deployment_schedule["daily_our"] = (deployment_schedule["tariff"] *
                                    deployment_schedule["grid_kw"])

daily_comparison = (deployment_schedule
                    .resample("D")
                    .agg({"daily_grid_only": "sum",
                          "daily_our": "sum"}))
daily_comparison["daily_saving"] = (daily_comparison["daily_grid_only"] -
                                    daily_comparison["daily_our"])

example_day = daily_comparison.index[0].strftime("%Y-%m-%d")
grid_cost   = daily_comparison.loc[daily_comparison.index[0], "daily_grid_only"]
our_cost    = daily_comparison.loc[daily_comparison.index[0], "daily_our"]
saving      = daily_comparison.loc[daily_comparison.index[0], "daily_saving"]

print(f"Example day {example_day}")
print(f"Grid-only cost : {grid_cost:.2f} ₹")
print(f"Our cost       : {our_cost:.2f} ₹")
print(f"Money saved    : {saving:.2f} ₹")
print(f"Percentage saved that day: {(saving/grid_cost)*100:.1f} %")

display(daily_comparison.head())

Example day 2025-10-26
Grid-only cost : 107.20 ₹
Our cost       : 81.54 ₹
Money saved    : 25.66 ₹
Percentage saved that day: 23.9 %


,daily_grid_only,daily_our,daily_saving
timestamp,,,
2025-10-26 00:00:00+05:30,107.2,81.54375,25.65625
